In [ ]:
%env JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
%env FLINK_CONF_DIR=/Users/lgfquentin/dev/medical-dashboard/back-end-synthea-flink/config

In [ ]:
%load_ext streaming_jupyter_integrations.magics

In [ ]:
%flink_connect --execution-target remote --remote-hostname localhost --remote-port 8081
#%flink_connect 

In [ ]:
%%flink_execute_sql
CREATE TABLE patient (
  `id` STRING,
  `name` ROW(`family` STRING, `given` STRING ARRAY) ARRAY,
  `gender` STRING,
  `birthDate` STRING,
  `telecom` ROW ( `value` STRING ) ARRAY,
  `address` ROW (
    `line` STRING ARRAY,
    `city` STRING,
    `state` STRING,
    `postalCode` STRING,
    `country` STRING
  ) ARRAY,
  `maritalStatus` ROW (
    `text` STRING
  )
) WITH (
  'connector' = 'kafka',
  'topic' = 'Patient',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE TABLE diagnosticReport (
  `id` STRING,
  `subject` ROW (`reference` STRING),
  `code` ROW(`coding` ROW(`display` STRING) ARRAY)
) WITH (
  'connector' = 'kafka',
  'topic' = 'DiagnosticReport',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE OR REPLACE TABLE hospital (
  `id` STRING,
  `identifier` ROW (`value` STRING) ARRAY,
  `status` STRING,
  `name` STRING,
  `telecom` ROW (`system` STRING, `value` STRING) ARRAY,
  `address` ROW (
    `line` STRING ARRAY,
    `city` STRING,
    `state` STRING,
    `postalCode` STRING,
    `country` STRING
  ),
  `position` ROW (
    `longitude` DOUBLE,
    `latitude` DOUBLE
  ),
  `managingOrganization` ROW (
    `identifier` ROW (`value` STRING)
  )
) WITH (
  'connector' = 'kafka',
  'topic' = 'Location',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE TABLE practitionerRole (
  `id` STRING,
  `practitioner` ROW (
    `identifier` ROW (
      `value` STRING
    )
  ),
  `organization` ROW (
    `identifier` ROW (
      `value` STRING
    )
  ),
  `specialty` ROW (
    `text` STRING
  ) ARRAY
) WITH (
  'connector' = 'kafka',
  'topic' = 'PractitionerRole',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE TABLE practitioner (
  `identifier` ROW (
    `value` STRING
  ) ARRAY,
  `active` BOOLEAN,
  `name` ROW (
    `family` STRING,
    `given` STRING ARRAY
  ) ARRAY,
  `telecom` ROW (
    `value` STRING
  ) ARRAY,
  `address` ROW (
    `line` STRING ARRAY,
    `city` STRING,
    `state` STRING,
    `postalCode` STRING,
    `country` STRING
  ) ARRAY,
  `gender` STRING
) WITH (
  'connector' = 'kafka',
  'topic' = 'Practitioner',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE TABLE encounter (
  `id` STRING,
  `status` STRING,
  `type` ROW (
    `coding` ROW (
      `code` STRING,
      `display` STRING
    ) ARRAY  ) ARRAY,
  `subject` ROW (
    `reference` STRING,
    `display` STRING
  ),
  `participant` ROW (
    `individual` ROW (
      `reference` STRING,
      `display` STRING
    )
  ) ARRAY,
  `period` ROW (
    `start` STRING,
    `end` STRING
  ),
  `location` ROW (
    `location` ROW (
      `reference` STRING,
      `display` STRING
    )
  ) ARRAY
) WITH (
  'connector' = 'kafka',
  'topic' = 'Encounter',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

## Insert into Postgres

Create table in Postgres (replace container name by yours)

```bash
docker exec -it flink-sql_tutorial-db-1 psql -U example
> create table patient(identifier VARCHAR, familyName VARCHAR, gender VARCHAR, diagnosticReportLabel VARCHAR, PRIMARY KEY(identifier));
```

In [ ]:
%%flink_execute_sql
CREATE TABLE hospital_psql (
  `id_organization` STRING,
  `id_location` STRING,
  `status` STRING,
  `name` STRING,
  `telecom` STRING,
  `address_line` STRING,
  `city` STRING,
  `state` STRING,
  `postalCode` STRING,
  `country` STRING,
  `longitude` DOUBLE,
  `latitude` DOUBLE,
  PRIMARY KEY (`id_organization`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://db:5432/example',
  'table-name' = 'public.hospital',
  'username' = 'example',
  'password' = 'example'
);


In [ ]:
%%flink_execute_sql
CREATE TABLE practitioner_psql (
  `identifier` STRING,
  `name` STRING,
  `email` STRING,
  `adress` STRING,
  `city` STRING,
  `state` STRING,
  `postalCode` STRING,
  `country` STRING,
  `gender` STRING,
  `role` STRING,
  `id_organization` STRING,
  PRIMARY KEY (`identifier`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://db:5432/example',
  'table-name' = 'public.practitioner',
  'username' = 'example',
  'password' = 'example'
);


In [ ]:
%%flink_execute_sql
CREATE TABLE patient_psql (
  `identifier` STRING,
  `surname` STRING,
  `familyName` STRING,
  `gender` STRING,
  `birthDate` STRING,
  `telecom` STRING,
  `adress` STRING,
  `city` STRING,
  `state` STRING,
  `postalCode` STRING,
  `country` STRING,
  `maritalStatus` STRING,
  PRIMARY KEY (`identifier`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://db:5432/example',
  'table-name' = 'public.patient',
  'username' = 'example',
  'password' = 'example'
);


In [ ]:
%%flink_execute_sql
CREATE TABLE visit_psql (
  `identifier` STRING,
  `id_organization` STRING,
  `id_practitioner` STRING,
  `id_patient` STRING,
  `status` STRING,
  `startTime` TIMESTAMP,
  `endTime` TIMESTAMP,
  `type` STRING,
  PRIMARY KEY (`identifier`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://db:5432/example',
  'table-name' = 'public.visit',
  'username' = 'example',
  'password' = 'example'
);


In [ ]:
%%flink_execute_sql
INSERT INTO patient_psql
SELECT id, p.name[1].family as familyname, p.name[1].given[1] as surname, gender, birthDate, p.telecom[1][1] as telecom, p.address[1].line[1] as address,  p.address[1].city as city, p.address[1].state as state, p.address[1].country as country, p.address[1].postalCode as postalCode, maritalStatus.text as maritalStatus from patient p;

In [ ]:
%%flink_execute_sql
INSERT INTO hospital_psql
SELECT 
  h.managingOrganization.identifier[1] as id_organization,
  h.id as id_location, 
  status, 
  name, 
  h.telecom[1][1] AS telecom, 
  h.address.line[1] AS address,  
  h.address.city AS city, 
  h.address.state AS state, 
  h.address.country AS country, 
  h.address.postalCode AS postalCode, 
  h.`position`.longitude AS longitude,
  h.`position`.latitude AS latitude
  FROM 
  hospital h WHERE h.managingOrganization.identifier[1] <>'None';


In [ ]:
%%flink_execute_sql
SELECT * FROM hospital WHERE managingOrganization = 'None';

In [29]:
%%flink_execute_sql
SELECT 
  h.managingOrganization.identifier[1] as id_organization,
  h.id as id_location, 
  status, 
  name, 
  h.telecom[1][1] AS telecom, 
  h.address.line[1] AS address,  
  h.address.city AS city, 
  h.address.state AS state, 
  h.address.country AS country, 
  h.address.postalCode AS postalCode, 
  h.`position`.longitude AS longitude,
  h.`position`.latitude AS latitude
  FROM 
  hospital h WHERE h.managingOrganization.identifier[1] <>'None';

This job runs in a background, please either wait or interrupt its execution before continuing


Job starting...
Job started
Pulling query results...
Results will be pulled from the job. You can interrupt any time to show partial results.
Execution result will bind to `execution_result` variable.


IntText(value=0, description='Loaded rows: ')

,id_organization,id_location,status,name,telecom,address,city,state,country,postalCode,longitude,latitude
0,3ff0f58b-7a1f-3334-9449-ff15f68b7d86,69fc5bac-35b9-3671-9b39-e157b58b6207,active,BETH ISRAEL DEACONESS HOSPITAL PLYMOUTH INC,phone,275 SANDWICH ST,PLYMOUTH,MA,US,023607761,-70.645537,41.941866
1,7b20d1c4-c56b-3647-801c-5e4e13679073,c64ea29a-9c02-31a6-9721-2c245c1c085d,active,BETH ISRAEL DEACONESS HOSPITAL PLYMOUTH INC,phone,275 SANDWICH ST,PLYMOUTH,MA,US,023602183,-70.645537,41.941866
2,3ff0f58b-7a1f-3334-9449-ff15f68b7d86,69fc5bac-35b9-3671-9b39-e157b58b6207,active,BETH ISRAEL DEACONESS HOSPITAL PLYMOUTH INC,phone,275 SANDWICH ST,PLYMOUTH,MA,US,023607761,-70.645537,41.941866
3,7b20d1c4-c56b-3647-801c-5e4e13679073,c64ea29a-9c02-31a6-9721-2c245c1c085d,active,BETH ISRAEL DEACONESS HOSPITAL PLYMOUTH INC,phone,275 SANDWICH ST,PLYMOUTH,MA,US,023602183,-70.645537,41.941866
4,3ff0f58b-7a1f-3334-9449-ff15f68b7d86,69fc5bac-35b9-3671-9b39-e157b58b6207,active,BETH ISRAEL DEACONESS HOSPITAL PLYMOUTH INC,phone,275 SANDWICH ST,PLYMOUTH,MA,US,023607761,-70.645537,41.941866
5,7b20d1c4-c56b-3647-801c-5e4e13679073,c64ea29a-9c02-31a6-9721-2c245c1c085d,active,BETH ISRAEL DEACONESS HOSPITAL PLYMOUTH INC,phone,275 SANDWICH ST,PLYMOUTH,MA,US,023602183,-70.645537,41.941866
6,52ecddd8-42f6-35cc-88a5-cf474c02fb49,a8fa653f-3447-3452-b7a1-a282ef226112,active,LIFE CARE CENTER OF PLYMOUTH,phone,94 OBERY STREET,PLYMOUTH,MA,US,023602130,-70.655032,41.937250
7,e69e7cbb-8362-30d1-83f2-3a0c492fe2b1,b36746ab-80dd-321d-80d2-ccb8baeb3121,active,"PLYMOUTH CARVER PRIMARY CARE, P.C.",phone,110 LONG POND RD,PLYMOUTH,MA,US,023602642,-70.655143,41.925583
8,52ecddd8-42f6-35cc-88a5-cf474c02fb49,a8fa653f-3447-3452-b7a1-a282ef226112,active,LIFE CARE CENTER OF PLYMOUTH,phone,94 OBERY STREET,PLYMOUTH,MA,US,023602130,-70.655032,41.937250
9,e69e7cbb-8362-30d1-83f2-3a0c492fe2b1,b36746ab-80dd-321d-80d2-ccb8baeb3121,active,"PLYMOUTH CARVER PRIMARY CARE, P.C.",phone,110 LONG POND RD,PLYMOUTH,MA,US,023602642,-70.655143,41.925583
